## 3.4 Regional Planning
The SOUTHERN CONFEDERATION OF KIBBUTZIM is a group of three kibbutzim (communal farming communities) in Israel. Overall planning for this group is done in its Coordinating Technical Office. This office currently is planning agricultural production for the coming year.

The agricultural output of each kibbutz is limited by both the amount of available irrigable land and the quantity of water allocated for irrigation by the Water Commissioner (a national government official). These data are given in Table 3.8. The crops suited for this region include sugar beets, cotton, and sorghum, and these are the three being considered for the upcoming season. These crops differ primarily in their expected net return per acre and their consumption of water. In addition, the Ministry of Agriculture has set a maximum quota for the total acreage that can be devoted to each of these crops by the Southern Confederation of Kibbutzim, as shown in Table 3.9.

---

 **Formulation as a Linear Programming Problem**

The quantities to be decided upon are the number of acres to devote to each of the three crops at each of the three kibbutzim. The decision variables $x_j$ (j = 1, 2, . . . , 9) represent these nine quantities.

Since the measure of effectiveness $Z$ is the total net return, the resulting linear programming model for this problem is:

**Maximize**
$$Z = 1,000(x_1 + x_2 + x_3) + 750(x_4 + x_5 + x_6) + 250$$


**TABLE 3.10 Decision variables for the Southern Confederation of Kibbutzim problem**

| Crop | Kibbutz 1 | Kibbutz 2 | Kibbutz 3 | Net Return | Max Output | Water Consumption |
| :--- | :---: | :---: | :---: | :---: | :---: | :---: |
| Sugar beets | $x_1$ | $x_2$ | $x_3$ | $1,000$ | 600 | 3 |
| Cotton | $x_4$ | $x_5$ | $x_6$ | $750$ | 500 | 2 |
| Sorghum | $x_7$ | $x_8$ | $x_9$ | $250$ | 325 | 1 |
| Water Allocation | 600 | 800 | 350 |
| Usable Land | 400 | 600 | 300 |



In [46]:
using JuMP, HiGHS

model = Model(HiGHS.Optimizer)

@variable(model, x[1:9] >= 0);

In [47]:
# We want to maximize the total net return
@objective(model, Max, 1000 * (x[1] + x[2] + x[3]) + 750 * (x[4] + x[5] + x[6]) + 250 * (x[7] + x[8] + x[9]));

In [48]:
# Constraints for water allocation for each kibbutz
@constraint(model, 3 * x[1] + 2 * x[4] + 1 * x[7] <= 600) # Kibbutz 1
@constraint(model, 3 * x[2] + 2 * x[5] + 1 * x[8] <= 800) # Kibbutz 2
@constraint(model, 3 * x[3] + 2 * x[6] + 1 * x[9] <= 375) # Kibbutz 3

# Constraints for usable land
@constraint(model, x[1] + x[4] + x[7] <= 400) # Kibbutz 1
@constraint(model, x[2] + x[5] + x[8] <= 600) # Kibbutz 2
@constraint(model, x[3] + x[6] + x[9] <= 300) # Kibbutz 3

# Maximum output for each crop
@constraint(model, x[1] + x[2] + x[3] <= 600)
@constraint(model, x[4] + x[5] + x[6] <= 500)
@constraint(model, x[7] + x[8] + x[9] <= 325);

In [49]:
# Since proportion of each crop is 1:1:1, we can use the following constraint
@constraint(model, 3*(x[1] + x[4] + x[7]) - 2*(x[2] + x[5] + x[8]) == 0)
@constraint(model, 1*(x[2] + x[5] + x[8]) - 2*(x[3] + x[6] + x[9]) == 0)
@constraint(model, 4*(x[3] + x[6] + x[9]) - 3*(x[1] + x[4] + x[7]) == 0);

In [50]:
# Display the linear programming model with constraints
latex_formulation(model)

$$ \begin{aligned}
\max\quad & 1000 x_{1} + 1000 x_{2} + 1000 x_{3} + 750 x_{4} + 750 x_{5} + 750 x_{6} + 250 x_{7} + 250 x_{8} + 250 x_{9}\\
\text{Subject to} \quad & 3 x_{1} - 2 x_{2} + 3 x_{4} - 2 x_{5} + 3 x_{7} - 2 x_{8} = 0\\
 & x_{2} - 2 x_{3} + x_{5} - 2 x_{6} + x_{8} - 2 x_{9} = 0\\
 & -3 x_{1} + 4 x_{3} - 3 x_{4} + 4 x_{6} - 3 x_{7} + 4 x_{9} = 0\\
 & 3 x_{1} + 2 x_{4} + x_{7} \leq 600\\
 & 3 x_{2} + 2 x_{5} + x_{8} \leq 800\\
 & 3 x_{3} + 2 x_{6} + x_{9} \leq 375\\
 & x_{1} + x_{4} + x_{7} \leq 400\\
 & x_{2} + x_{5} + x_{8} \leq 600\\
 & x_{3} + x_{6} + x_{9} \leq 300\\
 & x_{1} + x_{2} + x_{3} \leq 600\\
 & x_{4} + x_{5} + x_{6} \leq 500\\
 & x_{7} + x_{8} + x_{9} \leq 325\\
 & x_{1} \geq 0\\
 & x_{2} \geq 0\\
 & x_{3} \geq 0\\
 & x_{4} \geq 0\\
 & x_{5} \geq 0\\
 & x_{6} \geq 0\\
 & x_{7} \geq 0\\
 & x_{8} \geq 0\\
 & x_{9} \geq 0\\
\end{aligned} $$

In [51]:
# Finally, we solve the model
optimize!(model)

Running HiGHS 1.11.0 (git hash: 364c83a51e): Copyright (c) 2025 HiGHS under MIT licence terms
LP   has 12 rows; 9 cols; 45 nonzeros
Coefficient ranges:
  Matrix [1e+00, 4e+00]
  Cost   [2e+02, 1e+03]
  Bound  [0e+00, 0e+00]
  RHS    [3e+02, 8e+02]
Presolving model
11 rows, 9 cols, 42 nonzeros  0s
Dependent equations search running on 3 equations with time limit of 1000.00s
Dependent equations search removed 1 rows and 6 nonzeros in 0.00s (limit = 1000.00s)
10 rows, 9 cols, 36 nonzeros  0s
Presolve : Reductions: rows 10(-2); columns 9(-0); elements 36(-9)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -5.9999767255e+03 Ph1: 10(39); Du: 9(5999.98) 0s
          6    -6.3333333333e+05 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model status        : Optimal
Simplex   iterations: 6
Objective value     :  6.3333333333e+05
P-D objective error :  9.1906760463e-17
HiGHS run time  

In [52]:
solution_summary(model, verbose=true)

solution_summary(; result = 1, verbose = true)
├ solver_name          : HiGHS
├ Termination
│ ├ termination_status : OPTIMAL
│ ├ result_count       : 1
│ ├ raw_status         : kHighsModelStatusOptimal
│ └ objective_bound    : 6.33333e+05
├ Solution (result = 1)
│ ├ primal_status        : FEASIBLE_POINT
│ ├ dual_status          : FEASIBLE_POINT
│ ├ objective_value      : 6.33333e+05
│ ├ dual_objective_value : 6.33333e+05
│ ├ relative_gap         : 1.83814e-16
│ └ value
│   ├ x[1] : 1.33333e+02
│   ├ x[2] : 1.00000e+02
│   ├ x[3] : 2.50000e+01
│   ├ x[4] : 1.00000e+02
│   ├ x[5] : 2.50000e+02
│   ├ x[6] : 1.50000e+02
│   ├ x[7] : 0.00000e+00
│   ├ x[8] : 0.00000e+00
│   └ x[9] : 0.00000e+00
└ Work counters
  ├ solve_time (sec)   : 8.95417e-04
  ├ simplex_iterations : 6
  ├ barrier_iterations : 0
  └ node_count         : -1